# Struct2XAS examples: from CIF/XYZ to XANES/EXAFS simulations with FDMNES and FEFF

- Authors: Beatriz G. Foschiani and Mauro Rovezzi
- Contact: mauro.rovezzi@esrf.fr
- Last modified: August 2023

Here we provide various examples on the usage of `larch.xrd.struct2xas` module.

## Example 1: Zn K-edge XANES of wurtzite ZnO with FDMNES from a CIF file

This example shows how to use the `Struct2XAS` class to convert a CIF file to a FDMNES XANES input. We will simulate Zn K-edge XAS of ZnO. 

- The input structure is taken from the Materials Project database: [mp-2133](https://legacy.materialsproject.org/materials/mp-2133/).
- The experimental data are taken from the SSHADE/FAME database: [DOI:10.26302/SSHADE/EXPERIMENT_ST_20180418_001](https://www.sshade.eu/data/EXPERIMENT_ST_20180418_001)

Import the main class and instantiate it with the CIF file and the name of the absorbing element

In [1]:
from larch.xrd.struct2xas import Struct2XAS
mat_obj = Struct2XAS(file = "../structuredata/struct2xas/ZnO_mp-2133.cif", abs_atom="Zn")

[struct2xas] INFO : Frames: 1, Absorbing sites: 1. (Indexes for frames and abs_sites start at 0)


to get the information about absorbing site

In [2]:
# information about absorbing site as pandas.DataFrame
mat_obj.get_abs_sites_info()

idx_abs,specie,frac_coords,wyckoff_site,cart_coords,occupancy_abs_atom,idx_struct
0,Zn,"[0.3333, 0.6667, 0.0005]",2b,[-0. 1.899 0.0029],1,1


the same, more programmatically

In [3]:
mat_obj.get_abs_sites()

[[0,
  'Zn',
  [0.3333, 0.6667, 0.0005],
  '2b',
  array([-0.    ,  1.899 ,  0.0029]),
  1,
  1]]

Getters and setters are available for changing the absorber site being considered. We can use the methods `set_abs_site` and `get_abs_site`. This structure has only one absorber site (an example with multiple sites is given later).

In [4]:
mat_obj.set_abs_site(0)

to get information on the coordination environment around absorber atom

In [5]:
mat_obj.get_coord_envs_info()

Coord. Env. from absorber atom: Zn at site 0
(For more details use get_coord_envs() method) 


(['T:4', 'T:5'],
   Element  Distance
 0     (O)   2.00419
 1     (O)   2.00424
 2     (O)   2.00424
 3     (O)   2.01243
 4     (O)   3.29439)

or programmatically

In [6]:
mat_obj.get_coord_envs()


[['Coord. Env. for Site 0',
  [{'ce_symbol': 'T:4',
    'ce_fraction': 0.8596848561093449,
    'csm': 0.0056914424517299726,
    'permutation': [0, 1, 2, 3]},
   {'ce_symbol': 'T:5',
    'ce_fraction': 0.14031514389065514,
    'csm': 4.062381866591052,
    'permutation': [1, 2, 4, 0, 3]}],
  [{'site': PeriodicSite: O (-0.0000, 1.8990, 2.0153) [0.3333, 0.6667, 0.3798],
    'index': 3,
    'image_cell': array([0, 0, 0])},
   {'site': PeriodicSite: O (1.6446, 0.9495, -0.6381) [0.6667, 0.3333, -0.1202],
    'index': 2,
    'image_cell': array([ 0,  0, -1])},
   {'site': PeriodicSite: O (-1.6446, 0.9495, -0.6381) [-0.3333, 0.3333, -0.1202],
    'index': 2,
    'image_cell': array([-1,  0, -1])},
   {'site': PeriodicSite: O (-0.0000, 3.7979, -0.6381) [0.6667, 1.3333, -0.1202],
    'index': 2,
    'image_cell': array([ 0,  1, -1])},
   {'site': PeriodicSite: O (-0.0000, 1.8990, -3.2915) [0.3333, 0.6667, -0.6202],
    'index': 3,
    'image_cell': array([ 0,  0, -1])}]]]

It is possible to visualize the local environment at a ginen radius from the absorber

In [7]:
mat_obj.visualize(radius=2.5)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

[struct2xas] INFO : {'Zn': 'red', 'O': 'green'}


To create the FDMNES input file using the default template in a temporary directory

In [3]:
mytemplate = None #: uses default FDMNES XANES template (-> `larch/xrd/templates/fdmnes.tmpl`)
mypath = None #: creates a default structure -> "mydir/fdmnes/input_structure/abs_atom/siteN/"
mat_obj.make_input_fdmnes(radius=7, green=False, template=mytemplate, parent_path=mypath)

[struct2xas] INFO : written FDMNES input -> /tmp/struct2xas-vw_vly9v/fdmnes/ZnO_mp-2133/Zn/frame0/site0/job_inp.txt


To show the created input file

In [4]:
! cat {mat_obj.outdir}/job_inp.txt

! FDMNES input file generated by Struct2XAS

header

comment
cif file name: ZnO_mp-2133
creation date:2023-06-22_0956

filout

job

range
-30.0 0.1 70.0 1.0 100 

radius
7

quadrupole

density

SCF



absorber
1

spgroup
186



crystal
3.28910296     3.28910296   5.30682100  90.00000000  90.00000000 120.00000000

30   0.33333333   0.66666667   0.00054800   Zn
 8   0.33333333   0.66666667   0.37976200    O

convolution

end


Run FDMNES (refer to FDMNES documentation if needed)

```python
#Example how to run FDMNES at the ESRF via SLURM 
input = mat_ojb.parent_path
! cd {input}; subfdmnes -c 30
```

The output files created by FDMNES will be:
- `job.txt`: non-convoluted spectra
- `job_conv.txt`: convoluted spectra (by fdmnes with default parameters)
- `job_out.txt`: FDMNES output during the execution of the program (NOTE: this is created by the SLURM submission script)
- `job_sd0.txt`: file with the projected density of states information
- `job_bav.txt`: file with large simulation infos (better to compress it or remove)

Please, refer to [XANES_Convolution.ipynb](./XANES_Convolution.ipynb) for a comparison of the simulated data with experimental one.

## Example 1a: Zn K-edge EXAFS with FEFF

We can also simulate the EXAFS spectrum with FEFF

In [4]:
mytemplate = None #: uses default FEFF EXAFS template (-> `larch/xrd/templates/feff_exafs.tmpl`)
mypath = mat_obj.parent_path #: if None creates a default structure -> "mydir/feff/input_structure/abs_atom/frameN/siteN/"
mat_obj.make_input_feff(radius=7, template=mytemplate, parent_path=mypath)

[struct2xas] INFO : written FEFF input -> /tmp/struct2xas-vw_vly9v/feff/ZnO_mp-2133/Zn/frame0/site0/feff.inp


to show the input file

In [5]:
! cat {mat_obj.outdir}/feff.inp

* FEFF8* input file generated by Struct2XAS

TITLE ZnO_mp-2133
TITLE 2023-06-22_1000
TITLE site 0

*  <central> <edge> edge energy = <eedge> eV
EDGE      K
S02       0.0

*         pot    xsph  fms   paths genfmt ff2chi
CONTROL   1      1     1     1     1      1
PRINT     1      0     0     0     0      0

*** ixc=0 means to use Hedin-Lundqvist
*         ixc  [ Vr  Vi ]
EXCHANGE  0

*         r_scf  [ l_scf   n_scf   ca ]
*SCF       5.0

RPATH     7
EXAFS     20
NLEG      6

* EXAFS damping, to mimic the structural disorder
* SIG2 0.005

* Debye-Waller factors
* DEBYE 0 0

*         emin  emax   eimag
*LDOS      -30   20     0.1


POTENTIALS

* ipot  Z   tag [lmax1 lmax2 xnatph sphinph]
    0  30 Zn(abs)
    2   8     O
    1  30    Zn 

ATOMS

*   x          y          z     ipot   tag    distance   occupancy
 -0.000001  -0.000036   0.000008 0  Zn(abs)    0.00000         *1 
 -0.000001   1.898928  -0.640982 2      O    2.00419         *1 
 -1.644552  -0.949518  -0.640982 2      O    

run FEFF

In [12]:
#via larch
from larch.xafs import FeffRunner
feff_inp = f"{mat_obj.parent_path}/feff.inp"
sim = FeffRunner(feff_inp)
sim.run()

 : ======== running Feff module feff8l_rdinp ========
 : Feff8L (EXAFS)      release  0.1
 : ZnO_mp-2133
 : 2023-06-14_1023
 : site 0
 : ======== running Feff module feff8l_pot ========
 : Calculating potentials ...
 : free atom potential and density for atom type    0
 : free atom potential and density for atom type    1
 : free atom potential and density for atom type    2
 : initial state energy
 : overlapped potential and density for unique potential    0
 : overlapped potential and density for unique potential    1
 : overlapped potential and density for unique potential    2
 : muffin tin radii and interstitial parameters
 : : ipot, Norman radius, Muffin tin radius, Overlap
 : 0  1.42792E+00  1.30600E+00  1.15000E+00
 : 1  1.41649E+00  1.30145E+00  1.15000E+00
 : 2  1.09462E+00  1.00452E+00  1.15000E+00
 : : mu_old=    -0.830
 : Done with module 1: potentials.
 : ======== running Feff module feff8l_xsph ========
 : Calculating cross-section and phases...
 : absorption cross secti

Note: can also be run via direct call to `feff8l`

```
! cd {mat_obj.parent_path}; feff8l
```

load the simulated EXAFS

In [13]:
from glob import glob
xmu = glob(f"{mat_obj.parent_path}/xmu.dat")[0]
from larch.io import read_ascii
gsim = read_ascii(xmu, labels=["energy", "erel", "k", "mu", "mu0", "chi"])

Load experimental data and quickly extract the EXAFS signal

In [14]:
gexp = read_ascii("../fdmnes/ZnO_SSHADE.data.txt", labels=["energy", "mu"])
from larch.xafs import pre_edge, autobk
pre_edge(gexp, e0=9661)
autobk(gexp)

then plot

In [18]:
from larch.plot.plotly_xafsplots import PlotlyFigure
fig = PlotlyFigure()
fig.add_plot(gsim.k+0.3, gsim.chi*gsim.k**2, label="FEFF simulation")
fig.add_plot(gexp.k, gexp.chi*gexp.k**2, label="EXP data")

fig.set_style(title="Wurtzite ZnO EXAFS", width=800, height=500, xaxis_title="k (Å^-1)", yaxis_title="chi(k) * k^2" )
fig.set_xrange(0, 10)
fig.show()

*Note: the agreement is not very good because the experimental EXAFS data stops at 8 $\AA^{-1}$ and the structure parameters are not optimized.*

## Example 2: structures with multiple sites

Here an example when the input structure has multiple sites for a given absorbing element. The number of non-equivalent sites is found via the [SpaceGroupAnalyzer](https://pymatgen.org/pymatgen.symmetry.analyzer.html#pymatgen.symmetry.analyzer.SpacegroupAnalyzer) in Pymatgen, which is based on [Spglib](https://spglib.readthedocs.io/en/latest/).


In [6]:
from larch.xrd.struct2xas import Struct2XAS
mat_obj = Struct2XAS(file = "../structuredata/struct2xas/ZnO_mp-997630.cif", abs_atom="Zn")

[struct2xas] INFO : Frames: 1, Absorbing sites: 4. (Indexes for frames and abs_sites start at 0)


In [7]:
mat_obj.get_abs_sites_info()

idx_abs,specie,frac_coords,wyckoff_site,cart_coords,occupancy_abs_atom,idx_struct
0,Zn,"[0.3333, 0.6667, 0.6651]",4f,[ 0. 1.7641 20.2245],1,3
1,Zn,"[0.3333, 0.6667, 0.0825]",4f,[0. 1.7641 2.5097],1,7
2,Zn,"[0.0, 0.0, 0.25]",2b,[0. 0. 7.6025],1,8
3,Zn,"[0.0, 0.0, 0.0]",2a,[0. 0. 0.],1,10


it is possible to select one site and get information on the local environment and visualize, as shown before

In [8]:
# analysis for second site
mat_obj.set_abs_site(2)
print(mat_obj.get_coord_envs_info())
mat_obj.visualize()

Coord. Env. from absorber atom: Zn at site 2
(For more details use get_coord_envs() method) 
(['T:6'],   Element  Distance
0     (O)   2.21121
1     (O)   2.21121
2     (O)   2.21121
3     (O)   2.21123
4     (O)   2.21123
5     (O)   2.21123)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

[struct2xas] INFO : {'Zn': 'red', 'O': 'green'}


to create the FDMNES inputs simply range over the sites

In [9]:
mypath = None
for site in range(mat_obj.nabs_sites): 
    mat_obj.set_abs_site(site)
    mat_obj.make_input_fdmnes(radius=7, green=False, parent_path=mypath)
    mypath = mat_obj.parent_path

[struct2xas] INFO : written FDMNES input -> /tmp/struct2xas-yzlt1g7s/fdmnes/ZnO_mp-997630/Zn/frame0/site0/job_inp.txt
[struct2xas] INFO : written FDMNES input -> /tmp/struct2xas-yzlt1g7s/fdmnes/ZnO_mp-997630/Zn/frame0/site1/job_inp.txt
[struct2xas] INFO : written FDMNES input -> /tmp/struct2xas-yzlt1g7s/fdmnes/ZnO_mp-997630/Zn/frame0/site2/job_inp.txt
[struct2xas] INFO : written FDMNES input -> /tmp/struct2xas-yzlt1g7s/fdmnes/ZnO_mp-997630/Zn/frame0/site3/job_inp.txt


In [10]:
for site in range(mat_obj.nabs_sites): 
    mat_obj.set_abs_site(site)
    mat_obj.make_input_feff(radius=7, parent_path=mypath)

[struct2xas] INFO : written FEFF input -> /tmp/struct2xas-yzlt1g7s/feff/ZnO_mp-997630/Zn/frame0/site0/feff.inp
[struct2xas] INFO : written FEFF input -> /tmp/struct2xas-yzlt1g7s/feff/ZnO_mp-997630/Zn/frame0/site1/feff.inp
[struct2xas] INFO : written FEFF input -> /tmp/struct2xas-yzlt1g7s/feff/ZnO_mp-997630/Zn/frame0/site2/feff.inp
[struct2xas] INFO : written FEFF input -> /tmp/struct2xas-yzlt1g7s/feff/ZnO_mp-997630/Zn/frame0/site3/feff.inp


## Example 3: grab CIFs from Materials Project

In this example some CIFs files are grabbed from the Materials Project database (MP) via a query and the inputs files are generated.

A personal API key is required to query the database (-> [here](https://legacy.materialsproject.org/open). *NOTE* the legacy version is used in this example, that is, the "old" version of MP. Export `your_api_key` to the environmental variable `MP_API_KEY`.



In [1]:
import os
from pymatgen.ext.matproj import _MPResterLegacy
from larch.xrd.struct2xas import Struct2XAS, save_cif_from_mp

api_key = os.getenv("MP_API_KEY")

we search for materials containing 2 elements, Zn and O, plus further filter those IDs not present in the ICSD database

In [2]:
# criterias to query the materials
entries = _MPResterLegacy(api_key).query(criteria={"elements": ("Zn", "O"), "nelements": {"$gte": 2}}, properties=["material_id", "icsd_ids"])
mp_ids = [e['material_id'] for e in entries if len(e['icsd_ids']) >= 1]  #: skip materials not present in the ICSD database
ids = mp_ids[0:10]

save the corresponding CIF files

In [3]:
cifs_path = None
for id in ids:
    try:
        cifs_path, cif_fname = save_cif_from_mp(api_key, id, parent_path=cifs_path)
    except Exception:
        print(f"ERROR: cannot retrieve {id}")
print(f"CIF files saved in: {cifs_path}")


[struct2xas] INFO : mp-2229 -> ZnO_mp-2229.cif
[struct2xas] INFO : mp-8484 -> ZnO2_mp-8484.cif
[struct2xas] INFO : mp-2133 -> ZnO_mp-2133.cif
[struct2xas] INFO : mp-1017539 -> ZnO_mp-1017539.cif
[struct2xas] INFO : mp-13161 -> ZnO_mp-13161.cif
[struct2xas] INFO : mp-1986 -> ZnO_mp-1986.cif
CIF files saved in: /tmp/mp-cifs-_rxzb_hq


load the CIF files and generate the FDMNES/FEFF inputs

In [8]:
from glob import glob
import tempfile

parent_path = tempfile.mkdtemp(prefix="struct2xas_ex3_")
cif_files = glob(f"{cifs_path}/*")

for cf in cif_files:
    s2x = Struct2XAS(cf, abs_atom="Zn")
    for abs_site in s2x.get_abs_sites():
        s2x.set_abs_site(abs_site[0])
        s2x.make_input_fdmnes(radius=7, parent_path=parent_path)
        s2x.make_input_feff(radius=7, parent_path=parent_path)


[struct2xas] INFO : Frames: 1, Absorbing sites: 1. (Indexes for frames and abs_sites start at 0)
[struct2xas] INFO : written FDMNES input -> /tmp/struct2xas_ex3_j58c_vyn/fdmnes/ZnO_mp-2229/Zn/frame0/site0/job_inp.txt


[struct2xas] INFO : written FEFF input -> /tmp/struct2xas_ex3_j58c_vyn/feff/ZnO_mp-2229/Zn/frame0/site0/feff.inp
[struct2xas] INFO : Frames: 1, Absorbing sites: 1. (Indexes for frames and abs_sites start at 0)
[struct2xas] INFO : written FDMNES input -> /tmp/struct2xas_ex3_j58c_vyn/fdmnes/ZnO2_mp-8484/Zn/frame0/site0/job_inp.txt
[struct2xas] INFO : written FEFF input -> /tmp/struct2xas_ex3_j58c_vyn/feff/ZnO2_mp-8484/Zn/frame0/site0/feff.inp
[struct2xas] INFO : Frames: 1, Absorbing sites: 1. (Indexes for frames and abs_sites start at 0)
[struct2xas] INFO : written FDMNES input -> /tmp/struct2xas_ex3_j58c_vyn/fdmnes/ZnO_mp-1017539/Zn/frame0/site0/job_inp.txt
[struct2xas] INFO : written FEFF input -> /tmp/struct2xas_ex3_j58c_vyn/feff/ZnO_mp-1017539/Zn/frame0/site0/feff.inp
[struct2xas] INFO : Frames: 1, Absorbing sites: 1. (Indexes for frames and abs_sites start at 0)
[struct2xas] INFO : written FDMNES input -> /tmp/struct2xas_ex3_j58c_vyn/fdmnes/ZnO_mp-2133/Zn/frame0/site0/job_inp.txt
[s

the next steps consist in running the simulations and comparing with experimental data. This is beyond the scope of current example.

## Example 4: from XYZ (single or multi-frame) to XAS

This example shows the case of XYZ files as input. In particular, in the case the XYZ file contains multiple frames. This is a typical output format of molecular dynamics simulations.

Here the case of Ga K-edge of a GaBr solution is taken as example:
- Cécile Da Silva, Olivier Proux, Jean-Louis Hazemann, Julianne James-Smith, Denis Testemale, Toshio Yamaguchi,
X-ray absorption spectroscopy study of solvation and ion-pairing in aqueous gallium bromide solutions at supercritical conditions,
Journal of Molecular Liquids, Volume 147, Issues 1–2, 2009, Pages 83-95, ISSN 0167-7322, [https://doi.org/10.1016/j.molliq.2008.06.022]
- The experimental data are taken from the SSHADE database [DOI:10.26302/SSHADE/EXPERIMENT_ST_20180418_001](https://www.sshade.eu/data/EXPERIMENT_ST_20180418_001)
- The input structure is manually generated by simply changing the Ga-O distance



In [14]:
from larch.xrd.struct2xas import Struct2XAS
gabr = Struct2XAS(file="../structuredata/struct2xas/GaBr_multi-frame.xyz", abs_atom="Ga")

[struct2xas] INFO : Frames: 5, Absorbing sites: 1. (Indexes for frames and abs_sites start at 0)


it is possible to select a given frame, visualize and get the coordination environment

In [84]:
gabr.set_frame(2)
gabr.visualize(5)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

[struct2xas] INFO : {'Ga': 'red', 'O': 'green', 'H': 'blue'}


In [85]:
gabr.get_coord_envs_info()

Coord. Env. from absorber atom: Ga at site 0
(For more details use get_coord_envs() method) 


("Elements Dict = {'O': 6}",
   Element  Distance
 0     (O)      1.96
 1     (O)      1.96
 2     (O)      1.96
 3     (O)      1.96
 4     (O)      1.96
 5     (O)      1.96)

generate FEFF EXAFS inputs (for FDMNES is equivalent)

In [73]:
import tempfile
outdir = tempfile.mkdtemp(prefix="gabr-")

for frame in range(gabr.nframes):
    gabr.set_frame(frame)
    gabr.make_input_feff(radius=5, sig2=0.005, parent_path=outdir)

[struct2xas] INFO : written FEFF input -> /tmp/gabr-io7plr_g/feff/GaBr_multi-frame/Ga/frame0/site0/feff.inp
[struct2xas] INFO : written FEFF input -> /tmp/gabr-io7plr_g/feff/GaBr_multi-frame/Ga/frame1/site0/feff.inp
[struct2xas] INFO : written FEFF input -> /tmp/gabr-io7plr_g/feff/GaBr_multi-frame/Ga/frame2/site0/feff.inp
[struct2xas] INFO : written FEFF input -> /tmp/gabr-io7plr_g/feff/GaBr_multi-frame/Ga/frame3/site0/feff.inp
[struct2xas] INFO : written FEFF input -> /tmp/gabr-io7plr_g/feff/GaBr_multi-frame/Ga/frame4/site0/feff.inp


run the simulations

In [74]:
from glob import glob
from larch.xafs import FeffRunner

feffinps = glob(f"{outdir}/**/feff.inp", recursive=True)
for feffinp in feffinps:
    sim = FeffRunner(feffinp)
    sim.run()

 : ======== running Feff module feff8l_rdinp ========
 : Feff8L (EXAFS)      release  0.1
 : GaBr_multi-frame
 : 2023-08-03_1703
 : site 0
 : ======== running Feff module feff8l_pot ========
 : Calculating potentials ...
 : free atom potential and density for atom type    0
 : free atom potential and density for atom type    1
 : free atom potential and density for atom type    2
 : initial state energy
 : overlapped potential and density for unique potential    0
 : overlapped potential and density for unique potential    1
 : overlapped potential and density for unique potential    2
 : muffin tin radii and interstitial parameters
 : : ipot, Norman radius, Muffin tin radius, Overlap
 : 0  1.42868E+00  1.27476E+00  1.15000E+00
 : 1  1.17353E+00  9.97844E-01  1.15000E+00
 : 2  1.03739E+00  8.59043E-01  1.15000E+00
 : : mu_old=     1.982
 : Done with module 1: potentials.
 : ======== running Feff module feff8l_xsph ========
 : Calculating cross-section and phases...
 : absorption cross 

sometimes is useful to export all the `xmu.dat` files in a single HDF5 file container

In [62]:
import os
import h5py
from larch.io import read_ascii

h5dir = os.path.join(outdir, "hdf5")
try:
    os.makedirs(h5dir)
except FileExistsError:
    pass
h5out = os.path.join(h5dir, "gabr_exafs.h5")

xmus = glob(f"{outdir}/**/xmu.dat", recursive=True)
labels = ["energy", "erel", "k", "mu", "mu0", "chi"]

with h5py.File(h5out, "w") as f:
    for xmu in xmus:
        sample, abs_at, framestr, sitestr = xmu.split(outdir)[-1].split(os.sep)[2:-1]
        gxmu = read_ascii(xmu, labels=labels)
        grouppath = f"/{sample}/{abs_at}/{framestr}/{sitestr}"
        try:
            f.create_group(grouppath)
        except Exception:
            pass
        for lab in labels:
            f.create_dataset(f"{grouppath}/{lab}", data=getattr(gxmu, lab))
print(f"xmu data exported to HDF5: {h5out}")

xmu data exported to HDF5: /tmp/gabr-faqmx65_/hdf5/gabr_exafs.h5


In [75]:
from larch.xafs import pre_edge, autobk
from larch.plot.plotly_xafsplots import PlotlyFigure, plotlabels

fig = PlotlyFigure()

xmus = glob(f"{outdir}/**/xmu.dat", recursive=True)
for xmu in xmus:
    sample, abs_at, framestr, sitestr = xmu.split(outdir)[-1].split(os.sep)[2:-1]
    gxmu = read_ascii(xmu, labels=["energy", "erel", "k", "mu", "mu0", "chi"])
    fig.add_plot(x=gxmu.k, y=gxmu.chi*gxmu.k**2, label=f"{sample}/{framestr}")

#EXP DATA
gexp = read_ascii("../structuredata/struct2xas/GaBr_GaK_SSHADE.data.txt", labels=["energy", "mu"])
pre_edge(gexp)
autobk(gexp)
fig.add_plot(x=gexp.k, y=gexp.chi*gexp.k**2, color="black", label="exp data")



fig.set_style(title="GaBr (Ga K-edge): FEFF sim. vs exp.", width=900, height=500, xaxis_title=plotlabels.k, yaxis_title=plotlabels.chi2)
fig.set_xrange(2, 14)
fig.set_yrange(-3, 3)
fig.show()

In [64]:
with h5py.File(h5out, "r") as f:
    print(f.keys())

<KeysViewHDF5 ['GaBr_multi-frame']>
